In [25]:
import numpy as np
import pandas as pd
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split 
import shap

In [2]:
df = pd.read_csv('GKFlagMarginData.csv')

In [3]:
#df = df.drop(['index'], axis=1)

In [4]:
df

,GKFlag,Age,Prof,Det,Amb,Mins,Av Rat,Imp M,Pres,Cons,Ada,Inj Pr,WR,CA,Margin,Training facilities,Div Rep,Growth
0,0,34.061602,7,8,11,51,6.70,11,8,10,10,14,3000,92,30,10,107,-11
1,0,41.494867,18,17,8,4380,6.84,14,13,6,14,1,2400,70,50,3,46,-10
2,1,36.024641,12,13,11,540,6.77,13,12,13,9,5,2000,73,39,6,85,-6
3,0,35.244353,18,15,12,3703,7.21,13,16,12,10,14,1250,69,36,3,46,-4
4,0,35.331964,15,16,15,3837,6.56,11,12,12,17,6,2900,103,25,12,107,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30251,0,16.388775,10,13,10,0,0.00,5,11,7,19,11,50,31,57,5,107,1
30252,0,16.205339,14,10,9,0,0.00,4,11,6,7,12,50,33,60,5,107,4
30253,0,16.265572,12,13,11,0,0.00,4,8,13,14,5,50,30,60,5,107,6
30254,0,16.232717,11,7,13,0,0.00,9,12,6,9,7,50,36,47,5,107,0


In [5]:
x = df.drop(['Growth','Ada','WR'], axis=1)
y = df['Growth']

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split( 
    x, y, test_size=0.2, random_state=2023)

In [7]:
xval, xtest, yval, ytest = train_test_split( 
    xtest, ytest, test_size=0.5, random_state=2023)

In [8]:
torch.set_default_dtype(torch.float32)
xtrain = torch.tensor(xtrain.values)
xval = torch.tensor(xval.values)
xtest = torch.tensor(xtest.values)
ytrain = torch.tensor(ytrain.values)
yval = torch.tensor(yval.values)
ytest = torch.tensor(ytest.values)

In [9]:
train_dataset = TensorDataset(xtrain, ytrain)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers = 1)

val_dataset = TensorDataset(xval, yval)

val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers = 1)

test_dataset = TensorDataset(xtest, ytest)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers = 1)

In [10]:
import torch
import torch.nn as nn

class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(len(xtrain[0]), 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 16)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(16, 1)
        #self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        bound = x[0][-3]
        x = self.fc1(x)
        x = self.relu(x)
        #x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        #x = self.dropout(x)
        x = self.fc3(x)
        return torch.min(x[0][0],bound).unsqueeze(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SimpleMLP().to(device)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [12]:
criterion = nn.MSELoss().to(device)

In [13]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device).float(), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)


        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device).float(), labels.to(device).float()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)


        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs},Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss

            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


In [14]:
model.load_state_dict(torch.load('best2(9.4flag).pth'))

<All keys matched successfully>

In [21]:
num_epochs = 1000
patience = 20

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience)

Epoch 1/1000,Train Loss: 8.3633, Val Loss: 9.2420
Epoch 2/1000,Train Loss: 8.3633, Val Loss: 9.2420
Epoch 3/1000,Train Loss: 8.3633, Val Loss: 9.2420


KeyboardInterrupt: 

In [15]:
with torch.no_grad():
    model.eval()
    test_loss = 0.0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device).float(), labels.to(device).float()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)


    test_loss /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')

Test Loss: 8.5849


In [16]:
model.eval()

SimpleMLP(
  (fc1): Linear(in_features=15, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)

In [41]:
xtest = xtest.to(device)

In [43]:
xtest_tensor = torch.unsqueeze(xtest.float(), 0)  # Add batch dimension
explainer = shap.DeepExplainer(model, xtest_tensor)

In [44]:
shap_values = explainer.shap_values(xtest_tensor)

RuntimeError: The size of tensor a (64) must match the size of tensor b (16) at non-singleton dimension 2

In [23]:
model(torch.tensor([[0,16,20,20,20,4000,7.3,20,20,20,1,100,100,20,185]]).to(device).float())

tensor([40.2960], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [18]:
model(torch.tensor([[1,35,1,1,1,0,0,1,1,1,20,100,100,1,46]]).to(device).float())

tensor([-11.9398], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [46]:
df

,GKFlag,Age,Prof,Det,Amb,Mins,Av Rat,Imp M,Pres,Cons,Ada,Inj Pr,WR,CA,Margin,Training facilities,Div Rep,Growth
0,0,34.061602,7,8,11,51,6.70,11,8,10,10,14,3000,92,30,10,107,-11
1,0,41.494867,18,17,8,4380,6.84,14,13,6,14,1,2400,70,50,3,46,-10
2,1,36.024641,12,13,11,540,6.77,13,12,13,9,5,2000,73,39,6,85,-6
3,0,35.244353,18,15,12,3703,7.21,13,16,12,10,14,1250,69,36,3,46,-4
4,0,35.331964,15,16,15,3837,6.56,11,12,12,17,6,2900,103,25,12,107,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30251,0,16.388775,10,13,10,0,0.00,5,11,7,19,11,50,31,57,5,107,1
30252,0,16.205339,14,10,9,0,0.00,4,11,6,7,12,50,33,60,5,107,4
30253,0,16.265572,12,13,11,0,0.00,4,8,13,14,5,50,30,60,5,107,6
30254,0,16.232717,11,7,13,0,0.00,9,12,6,9,7,50,36,47,5,107,0
